In [1]:
%load_ext autoreload
%load_ext autotime

time: 401 µs (started: 2023-03-12 14:15:06 -07:00)


In [2]:
%run init.ipynb

time: 2.24 s (started: 2023-03-12 14:15:06 -07:00)


/usr/WS2/iopp/projects/vani-analysis-tool/virenv-lassen/lib/python3.7/site-packages/distributed/node.py:182: UserWarning: Port 8788 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39921 instead
  f"Port {expected} is already in use.\n"


In [3]:
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings('ignore')

import dask
import dask.array as da
import dask.bag as db
import dask.dataframe as dd
import json
import math
import numpy as np
import os
import pandas as pd
# import pyarrow as pa
from dask import compute, delayed
from dask.dataframe import DataFrame
from dask.distributed import Client, LocalCluster, fire_and_forget, wait, worker_client
from wisio.utils.file_utils import ensure_dir
from wisio.utils.json_encoders import NpEncoder
from wisio.utils.logger import create_logger, format_log

from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

time: 602 ms (started: 2023-03-12 14:15:09 -07:00)


In [4]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.LSFCluster
Dashboard: http://192.168.65.157:39921/status,
Dashboard: http://192.168.65.157:39921/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://192.168.65.157:35961,Workers: 0
Dashboard: http://192.168.65.157:39921/status,Total threads: 0
Started: Just now,Total memory: 0 B


time: 68.8 ms (started: 2023-03-12 14:15:09 -07:00)


In [5]:
import json

def load_global_min_max(log_dir: str):
    with open(f"{log_dir}/global.json") as file:
        global_min_max = json.load(file)
    return global_min_max

time: 1.16 ms (started: 2023-03-12 14:15:09 -07:00)


In [6]:
DELTA = 0.0001

time: 948 µs (started: 2023-03-12 14:15:09 -07:00)


In [7]:
log_dir = "/p/gpfs1/iopp/recorder_app_logs/mummi-wemul/nodes-32-ppn-32/_parquet"
ddf = dd.read_parquet(f"{log_dir}/*.parquet")
ddf 

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,proc_name,file_name,size,acc_pat,bandwidth,duration,tmid,file_id,proc_id
npartitions=914,,,,,,,,,,,,,,,,,,,,,
,int64,int64,int32,int32,int32,int32,float32,float32,object,int32,object,object,object,object,int64,int32,float32,float32,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


time: 664 ms (started: 2023-03-12 14:15:09 -07:00)


In [8]:
ddf.head()

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,...,app,proc_name,file_name,size,acc_pat,bandwidth,duration,tmid,file_id,proc_id
0,1,68024,0,22598272,2,0,3.027607,3.027607,MPI_Comm_size,0,...,wemul,,UNKNOWN,0,0,0.0,0.000000,30276069,0,0
1,2,68024,0,22598272,2,0,3.027650,3.027650,MPI_Comm_rank,0,...,wemul,,UNKNOWN,0,0,0.0,0.000000,30276498,0,0
2,3,68024,0,22598272,0,3,6.323863,6.324036,open,0,...,wemul,wemul#lassen83#68024#35184394687104,/dev/shm/job3087233017-26990-OMPI_COLL_IBM-0-c...,0,0,0.0,0.000173,63239492,3341432784288721016,-7155284101858470952
3,4,68024,0,22598272,0,3,6.324068,6.324075,open,0,...,wemul,wemul#lassen83#68024#35184394687104,/dev/shm/job3087233017-26990-OMPI_COLL_IBM-0-c...,0,0,0.0,0.000007,63240712,3341432784288721016,-7155284101858470952
4,5,68024,0,22598272,0,3,6.324091,6.324104,ftruncate,0,...,wemul,wemul#lassen83#68024#35184394687104,/dev/shm/job3087233017-26990-OMPI_COLL_IBM-0-c...,0,0,0.0,0.000013,63240972,3341432784288721016,-7155284101858470952


time: 12.6 s (started: 2023-03-12 14:15:10 -07:00)


In [11]:
ddf.index.count().compute()

897296255

time: 2min 20s (started: 2023-03-09 12:47:53 -08:00)


In [12]:
ddf['tend'].max().compute()

3722.03271484375

time: 24 s (started: 2023-03-09 12:50:15 -08:00)


In [13]:
ddf['file_id'].unique().count().compute()

2243274

time: 25 s (started: 2023-03-09 12:50:39 -08:00)


In [14]:
ddf['proc_id'].unique().count().compute()

14

time: 24.7 s (started: 2023-03-09 12:51:04 -08:00)


In [9]:
global_min_max = load_global_min_max(log_dir=log_dir)
global_min_max

{'file_id': [113104469769545699, 7805388507343687694],
 'proc_id': [-7155286342531755048, -7155284098314087464],
 'tmid': [0, 37220326429]}

time: 10.2 ms (started: 2023-03-12 14:15:23 -07:00)


In [10]:
from dask.distributed import Future, get_client
from typing import Union

DELTA_BINS = [
    0,
    0.001,
    0.01,
    0.1,
    0.25,
    0.5,
    0.75,
    1
]
DELTA_BIN_LABELS = [
    'none',
    'trivial',
    'very low',
    'low',
    'medium',
    'high',
    'very high',
    'critical'
]
TIME_PRECISION = 1e7

def set_metric_deltas(df: pd.DataFrame, metric: str, max_io_time: float):
    metric_col, csp_col, delta_col = (
        f"{metric}_sum",
        f"{metric}_csp",
        f"{metric}_delta",
    )
    df[csp_col] = df[metric_col].cumsum() / max_io_time
    df[delta_col] = df[csp_col].diff().fillna(df[csp_col])
    return df


def set_metric_percentages(df: pd.DataFrame, metric: str, max_io_time: float):
    metric_col, pero_col, perr_col = (
        f"{metric}_sum",
        f"{metric}_pero",
        f"{metric}_perr"
    )
    df[pero_col] = df[metric_col] / max_io_time
    df[perr_col] = df[metric_col] / df[metric_col].sum()
    return df


def set_metric_scores(df: pd.DataFrame, metric: str, col: str):
    bin_col, score_col, threshold_col = (
        f"{metric}_bin",
        f"{metric}_score",
        f"{metric}_th",
    )
    df[bin_col] = np.digitize(df[col], bins=DELTA_BINS, right=True)
    df[score_col] = np.choose(df[bin_col] - 1, choices=DELTA_BIN_LABELS, mode='clip')
    df[threshold_col] = np.choose(df[bin_col] - 1, choices=DELTA_BINS, mode='clip')
    return df.drop(columns=[bin_col])

def compute_tranges(global_min_max: dict, precision=TIME_PRECISION):
    tmid_min, tmid_max = global_min_max['tmid']
    tranges = np.arange(tmid_min, tmid_max, precision)
    return get_client().scatter(tranges)


def set_tranges(df: pd.DataFrame, tranges: Union[Future, np.ndarray]):
    tranges = tranges.result() if isinstance(tranges, Future) else tranges
    return df.assign(trange=np.digitize(df['tmid'], bins=tranges, right=True))

time: 4.22 ms (started: 2023-03-12 14:15:23 -07:00)


In [11]:
tranges = compute_tranges(global_min_max=global_min_max)
tranges

<Future: finished, type: numpy.ndarray, key: ndarray-3925ea0c6e2094c854465f077e9ef9cf>

time: 20.1 ms (started: 2023-03-12 14:15:23 -07:00)


In [12]:
VIEW_TYPES = ['trange', 'file_name', 'proc_name']

time: 630 µs (started: 2023-03-12 14:15:23 -07:00)


In [13]:
from typing import Iterable

def deepflatten(collection, ignore_types=(bytes, str)):
    for x in collection:
        if isinstance(x, Iterable) and not isinstance(x, ignore_types):
            yield from deepflatten(x)
        else:
            yield x

def unique_flatten():
    return dd.Aggregation(
        'unique_flatten',
        lambda s: s.apply(lambda x: np.unique(x).tolist()),
        lambda s0: s0.apply(lambda x: np.unique(x).tolist()),
        lambda s1: s1.apply(lambda x: np.unique(list(deepflatten(x))).tolist()),
    )

time: 1.91 ms (started: 2023-03-12 14:15:23 -07:00)


In [14]:
from copy import copy

view_types = copy(VIEW_TYPES)
view_types.append('io_cat')
view_types.append('acc_pat')
view_types.append('func_id')

main_view = ddf[(ddf['cat'] == 0) & (ddf['io_cat'].isin([1,2,3]))]  \
    .map_partitions(set_tranges, tranges=tranges) \
    .groupby(view_types) \
    .agg({
        'duration': [sum],
        'index': ['count'],
        'size': [min, max, sum],
#         'file_name': ['first'],
#         'proc_name': ['first'],
#         'func_id': [unique_flatten()],
    }) \
    .persist()

main_view


Dask DataFrame Structure:
              duration  index   size              
                   sum  count    min    max    sum
npartitions=1                                     
               float32  int64  int64  int64  int64
                   ...    ...    ...    ...    ...
Dask Name: aggregate-agg, 1 tasks

time: 190 ms (started: 2023-03-12 14:15:23 -07:00)


In [15]:
wait(main_view)

DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('aggregate-agg-266039f1403b1fff7416f5e71622bcf7', 0)>}, not_done=set())

time: 1min 44s (started: 2023-03-12 14:15:23 -07:00)


In [16]:
main_view = main_view.reset_index()
main_view.columns = ['_'.join(tup).rstrip('_') for tup in main_view.columns.values]
main_view

,trange,file_name,proc_name,io_cat,acc_pat,func_id,duration_sum,index_count,size_min,size_max,size_sum
npartitions=1,,,,,,,,,,,
,int64,object,object,int64,int64,object,float32,int64,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...


time: 60.9 ms (started: 2023-03-12 14:17:08 -07:00)


In [17]:
wait(main_view)

DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('aggregate-agg-266039f1403b1fff7416f5e71622bcf7', 0)>}, not_done=set())

time: 4.89 ms (started: 2023-03-12 14:17:08 -07:00)


In [18]:
!rm -rf {log_dir}/cached & mkdir -p {log_dir}/cached

time: 144 ms (started: 2023-03-12 14:17:08 -07:00)


In [19]:
main_view.repartition('100MB').to_parquet(f"{log_dir}/cached/main_view")

[None]

time: 8min 22s (started: 2023-03-12 14:17:08 -07:00)


In [27]:
!ls -lh {log_dir}/cached/main_view | head -n 10

total 6.4G
-rw------- 1 izzet iopp 2.0K Mar 12 14:25 _common_metadata
-rw------- 1 izzet iopp 155K Mar 12 14:25 _metadata
-rw------- 1 izzet iopp  55M Mar 12 14:19 part.0.parquet
-rw------- 1 izzet iopp  55M Mar 12 14:19 part.1.parquet
-rw------- 1 izzet iopp  55M Mar 12 14:20 part.10.parquet
-rw------- 1 izzet iopp  56M Mar 12 14:19 part.100.parquet
-rw------- 1 izzet iopp  56M Mar 12 14:25 part.101.parquet
-rw------- 1 izzet iopp  56M Mar 12 14:19 part.102.parquet
-rw------- 1 izzet iopp  55M Mar 12 14:18 part.103.parquet
time: 155 ms (started: 2023-03-12 14:34:26 -07:00)


In [ ]:
del main_view

In [29]:
main_view = dd.read_parquet(f"{log_dir}/cached/main_view/*.parquet")
main_view

,trange,file_name,proc_name,io_cat,acc_pat,func_id,duration_sum,index_count,size_min,size_max,size_sum
npartitions=119,,,,,,,,,,,
0,int64,object,object,int64,int64,object,float32,int64,int64,int64,int64
280889,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...
33144987,...,...,...,...,...,...,...,...,...,...,...
33425876,...,...,...,...,...,...,...,...,...,...,...


time: 207 ms (started: 2023-03-12 14:35:02 -07:00)


In [42]:
len(main_view)

33425877

time: 11.7 s (started: 2023-03-12 14:40:02 -07:00)


In [30]:
main_view['read_count'] = 0
main_view['write_count'] = 0
main_view['metadata_count'] = 0
main_view['read_count'] = main_view['read_count'].mask(main_view['io_cat'] == 1, main_view['index_count'])
main_view['write_count'] = main_view['write_count'].mask(main_view['io_cat'] == 2,  main_view['index_count'])
main_view['metadata_count'] = main_view['metadata_count'].mask(main_view['io_cat'] == 3,  main_view['index_count'])

main_view['read_size'] = 0
main_view['write_size'] = 0
main_view['metadata_size'] = 0
main_view['read_size'] = main_view['read_size'].mask(main_view['io_cat'] == 1, main_view['size_sum'])
main_view['write_size'] = main_view['write_size'].mask(main_view['io_cat'] == 2,  main_view['size_sum'])
main_view['metadata_size'] = main_view['metadata_size'].mask(main_view['io_cat'] == 3,  main_view['size_sum'])

main_view['read_time'] = 0
main_view['write_time'] = 0
main_view['metadata_time'] = 0
main_view['read_time'] = main_view['read_time'].mask(main_view['io_cat'] == 1, main_view['duration_sum'])
main_view['write_time'] = main_view['write_time'].mask(main_view['io_cat'] == 2,  main_view['duration_sum'])
main_view['metadata_time'] = main_view['metadata_time'].mask(main_view['io_cat'] == 3,  main_view['duration_sum'])

main_view['sequential_count'] = 0
main_view['random_count'] = 0
main_view['sequential_count'] = main_view['sequential_count'].mask(main_view['acc_pat'] == 0, main_view['index_count'])
main_view['random_count'] = main_view['random_count'].mask(main_view['acc_pat'] == 1, main_view['index_count'])

main_view['sequential_size'] = 0
main_view['random_size'] = 0
main_view['sequential_size'] = main_view['sequential_size'].mask(main_view['acc_pat'] == 0, main_view['size_sum'])
main_view['random_size'] = main_view['random_size'].mask(main_view['acc_pat'] == 1, main_view['size_sum'])

main_view['sequential_time'] = 0
main_view['random_time'] = 0
main_view['sequential_time'] = main_view['sequential_time'].mask(main_view['acc_pat'] == 0, main_view['duration_sum'])
main_view['random_time'] = main_view['random_time'].mask(main_view['acc_pat'] == 1, main_view['duration_sum'])


main_view['data_count'] = main_view['write_count'] + main_view['read_count']
main_view['data_size'] = main_view['write_size'] + main_view['read_size']
main_view['data_time'] = main_view['write_time'] + main_view['read_time']

main_view['open_time'] = 0
main_view['close_time'] = 0
main_view['seek_time'] = 0
main_view['stat_time'] = 0
main_view['open_time'] = main_view['open_time'].mask(main_view['func_id'].str.contains('open') & ~main_view['func_id'].str.contains('dir'),  main_view['duration_sum'])
main_view['close_time'] = main_view['close_time'].mask(main_view['func_id'].str.contains('close') & ~main_view['func_id'].str.contains('dir'),  main_view['duration_sum'])
main_view['seek_time'] = main_view['seek_time'].mask(main_view['func_id'].str.contains('seek'),  main_view['duration_sum'])
main_view['stat_time'] = main_view['stat_time'].mask(main_view['func_id'].str.contains('stat'),  main_view['duration_sum'])

# main_view['workload_type'] = 0
# main_view['workload_type'] = main_view['workload_type'].mask(main_view['read_count'] == 0, 3)
# main_view['workload_type'] = main_view['workload_type'].mask(main_view['write_count'] == 0, 1)

# agg_dict = {col: sum for col in main_view.columns}
# agg_dict['file_name_first'] = 'first'
# agg_dict['proc_name_first'] = 'first'
# agg_dict['func_id_unique_flatten'] = unique_flatten()
# for view_type in VIEW_TYPES:
#     agg_dict.pop(view_type)

final_view = main_view \
    .groupby(VIEW_TYPES) \
    .sum() \
    .persist()

final_view


,io_cat,acc_pat,duration_sum,index_count,size_min,size_max,size_sum,read_count,write_count,metadata_count,read_size,write_size,metadata_size,read_time,write_time,metadata_time,sequential_count,random_count,sequential_size,random_size,sequential_time,random_time,data_count,data_size,data_time,open_time,close_time,seek_time,stat_time
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,int64,float32,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


time: 762 ms (started: 2023-03-12 14:35:14 -07:00)


In [31]:
wait(final_view)

DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('dataframe-groupby-sum-agg-06dec51eacadcaba68bed42394c941a3', 0)>}, not_done=set())

time: 52.7 s (started: 2023-03-12 14:35:16 -07:00)


In [34]:
final_view.repartition('100MB').to_parquet(f"{log_dir}/cached/final_view")

[None]

time: 16.3 s (started: 2023-03-12 14:37:24 -07:00)


In [35]:
!ls -lh {log_dir}/cached/final_view | head -n 10

total 3.2G
-rw------- 1 izzet iopp 4.7K Mar 12 14:37 _common_metadata
-rw------- 1 izzet iopp  92K Mar 12 14:37 _metadata
-rw------- 1 izzet iopp 119M Mar 12 14:37 part.0.parquet
-rw------- 1 izzet iopp 120M Mar 12 14:37 part.1.parquet
-rw------- 1 izzet iopp 120M Mar 12 14:37 part.10.parquet
-rw------- 1 izzet iopp 120M Mar 12 14:37 part.11.parquet
-rw------- 1 izzet iopp 120M Mar 12 14:37 part.12.parquet
-rw------- 1 izzet iopp 120M Mar 12 14:37 part.13.parquet
-rw------- 1 izzet iopp 120M Mar 12 14:37 part.14.parquet
time: 2.31 s (started: 2023-03-12 14:37:41 -07:00)


In [38]:
del final_view

time: 700 µs (started: 2023-03-12 14:39:13 -07:00)


In [39]:
final_view = dd.read_parquet(f"{log_dir}/cached/final_view/*.parquet")
final_view

,io_cat,acc_pat,duration_sum,index_count,size_min,size_max,size_sum,read_count,write_count,metadata_count,read_size,write_size,metadata_size,read_time,write_time,metadata_time,sequential_count,random_count,sequential_size,random_size,sequential_time,random_time,data_count,data_size,data_time,open_time,close_time,seek_time,stat_time
npartitions=27,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,int64,float32,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


time: 1.26 s (started: 2023-03-12 14:39:21 -07:00)


In [40]:
final_view.head()

,,,io_cat,acc_pat,duration_sum,index_count,size_min,size_max,size_sum,read_count,write_count,metadata_count,...,random_size,sequential_time,random_time,data_count,data_size,data_time,open_time,close_time,seek_time,stat_time
file_name,proc_name,trange,,,,,,,,,,,,,,,,,,,,,
/dev/shm/job1704793069-26990-OMPI_COLL_IBM-0-collshm-comm4-master0,wemul#lassen83#169481#35184394687104,1,12,0,0.000017,4,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
/dev/shm/job1704793069-26990-OMPI_COLL_IBM-0-collshm-comm4-master1,wemul#lassen83#169481#35184394687104,1,12,0,0.000019,4,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
/dev/shm/job1704793069-26990-OMPI_COLL_IBM-0-collshm-comm4-master10,wemul#lassen83#169481#35184394687104,1,12,0,0.000016,4,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
/dev/shm/job1704793069-26990-OMPI_COLL_IBM-0-collshm-comm4-master11,wemul#lassen83#169481#35184394687104,1,12,0,0.000018,4,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
/dev/shm/job1704793069-26990-OMPI_COLL_IBM-0-collshm-comm4-master12,wemul#lassen83#169481#35184394687104,1,12,0,0.000017,4,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0


time: 792 ms (started: 2023-03-12 14:39:24 -07:00)


In [41]:
len(final_view)

9290294

time: 787 ms (started: 2023-03-12 14:39:46 -07:00)


In [43]:
max_io_time = final_view.groupby(['proc_name']).agg({'duration_sum':sum})['duration_sum'].max()
max_io_time

dd.Scalar<series-..., dtype=float32>

time: 69.6 ms (started: 2023-03-12 14:40:56 -07:00)


In [44]:
DELTA = 0.0001
CUT = 0.5

time: 805 µs (started: 2023-03-12 14:41:03 -07:00)


In [45]:
import itertools as it

def _view_permutations(r: int):
    return it.permutations(VIEW_TYPES, r + 1)

metric = 'duration'
views = {}
for view_permutation in it.chain.from_iterable(map(_view_permutations, range(len(VIEW_TYPES)))):
    # Read types
    parent_type = view_permutation[:-1]
    view_type = view_permutation[-1]
    # Get parent view
    parent_view = views[parent_type] if parent_type in views else final_view
    # Create colum names
    metric_col, delta_col = f"{metric}_sum", f"{metric}_delta"
    # Check view type
    if view_type is not 'proc_name':
        # Compute `proc_id` view first
        group_view = parent_view \
            .groupby([view_type, 'proc_name']) \
            .agg({metric_col: sum}) \
            .groupby([view_type]) \
            .max()
    else:
        # Compute group view
        group_view = parent_view \
            .groupby([view_type]) \
            .agg({metric_col: sum})
    # Set metric scores
    group_view = group_view \
        .map_partitions(set_metric_deltas, metric=metric, max_io_time=max_io_time) \
        .query(f"{delta_col} > @delta", local_dict={'delta': DELTA})
    # Find filtered records
    view = parent_view.query(f"{view_type} in @indices", local_dict={'indices': group_view.index.unique()})
    # Return views
    views[view_permutation] = view

len(views)

15

time: 767 ms (started: 2023-03-12 14:41:16 -07:00)


In [46]:
for view_key, view in views.items():
    print(view_key, len(view))

('trange',) 9268842
('file_name',) 19744
('proc_name',) 9290294
('trange', 'file_name') 19616
('trange', 'proc_name') 9268842
('file_name', 'trange') 14389
('file_name', 'proc_name') 19724
('proc_name', 'trange') 9268842
('proc_name', 'file_name') 19744
('trange', 'file_name', 'proc_name') 19601
('trange', 'proc_name', 'file_name') 19616
('file_name', 'trange', 'proc_name') 14377
('file_name', 'proc_name', 'trange') 14377
('proc_name', 'trange', 'file_name') 19616
('proc_name', 'file_name', 'trange') 14389
time: 2min 46s (started: 2023-03-12 14:41:26 -07:00)


In [65]:
import dask.dataframe as dd
import pandas as pd
from dask import compute, delayed
from dask.distributed import get_client
from typing import Any, Dict

BOTTLENECK_ORDER = dict(
    file_name=('file_name', 'proc_name', 'trange'),
    proc_name=('proc_name', 'trange', 'file_name'),
    trange=('trange', 'proc_name', 'file_name'),
)
BOTTLENECK_TYPE = dict(
    file_name='file_name',
    proc_name='proc_name',
    trange='trange'
)

def _calculate_llc(level_row: pd.Series):
    llc = dict(level_row)
    for view_type in VIEW_TYPES:
        if view_type in llc:
            llc.pop(view_type)
        bottleneck_type = BOTTLENECK_TYPE[view_type]
        if bottleneck_type in llc:
            llc.pop(bottleneck_type)
    return llc


@delayed
def _process_bottleneck_view(
    view_key: tuple,
    threshold: float,
    low_level_view: pd.DataFrame,
    mid_level_view: pd.DataFrame,
    high_level_view: pd.DataFrame,
):
    # Get view type
    view_type = view_key[-1]
    # Get ordered bottleneck columns
    _, ml_col, ll_col = BOTTLENECK_ORDER[view_type]
    # Init bottlenecks
    bottlenecks = {}
    # Loop through index tuples
    ids_tuple = low_level_view.index
    for hl_id, ml_id, ll_id in ids_tuple:
        hl_row = high_level_view.loc[hl_id]
        ml_row = mid_level_view.loc[(hl_id, ml_id)]
        ll_row = low_level_view.loc[(hl_id, ml_id, ll_id)]
        if hl_id not in bottlenecks:
            bottlenecks[hl_id] = {}
            bottlenecks[hl_id]['llc'] = _calculate_llc(hl_row)
            bottlenecks[hl_id][ml_col] = {}
        if ml_id not in bottlenecks[hl_id][ml_col]:
            bottlenecks[hl_id][ml_col][ml_id] = {}
            bottlenecks[hl_id][ml_col][ml_id]['llc'] = _calculate_llc(ml_row)
            bottlenecks[hl_id][ml_col][ml_id][ll_col] = {}
        if ll_id not in bottlenecks[hl_id][ml_col][ml_id][ll_col]:
            bottlenecks[hl_id][ml_col][ml_id][ll_col][ll_id] = {}
            bottlenecks[hl_id][ml_col][ml_id][ll_col][ll_id]['llc'] = _calculate_llc(ll_row)
    # Return view key & bottlenecks
    return view_key, threshold, bottlenecks

def detect_bottlenecks(views: Dict[tuple, dd.DataFrame], view_types: list) -> Dict[tuple, object]:
    # Keep bottleneck views
    bottleneck_views = {}
    # Run through views
    for view_key, view in views.items():
        # Generate bottleneck views
        bottleneck_views[view_key] = _generate_bottlenecks_views(
            view_key=view_key,
            view=view,
            view_types=view_types,
            max_io_time=max_io_time,
            metric=metric,
        )
    # Generate bottlenecks
    bottlenecks = _process_bottleneck_views(
        bottleneck_views=bottleneck_views,
        metric=metric,
    )
    # Return bottleneck views
    return bottlenecks, bottleneck_views

def _process_bottleneck_views(bottleneck_views: Dict[tuple, dd.DataFrame], metric: str):
    # Init bottlenecks
    bottlenecks = {}
    bottlenecks_delayed = []
    # Run through bottleneck views
    for view_key, view_dict in bottleneck_views.items():
        # For given thresholds
        for th in DELTA_BINS[1:-1]:  # [0.25, 0.5, 0.75]

            threshold_col = f"{metric}_th"

            low_level_view = view_dict['low_level_view']
            mid_level_view = view_dict['mid_level_view']
            high_level_view = view_dict['high_level_view']

            bottlenecks_delayed.append(_process_bottleneck_view(
                view_key=view_key,
                threshold=th,
                low_level_view=low_level_view.query(f"{threshold_col} >= @th", local_dict={'th': th}),
                mid_level_view=mid_level_view.query(f"{threshold_col} >= @th", local_dict={'th': th}),
                high_level_view=high_level_view.query(f"{threshold_col} >= @th", local_dict={'th': th})
            ))

    # Compute all bottlenecks
    results = compute(*bottlenecks_delayed)
    for view_key, th, result in results:
        bottlenecks[view_key] = bottlenecks[view_key] if view_key in bottlenecks else {}
        bottlenecks[view_key][f"{th:.3f}"] = result
    # Return all bottlenecks
    return bottlenecks

def _generate_bottlenecks_views(
    view_key: tuple,
    view: dd.DataFrame,
    view_types: list,
    max_io_time: dd.core.Scalar,
    metric: str,
):
    # Get view type
    view_type = view_key[-1]
    bottleneck_type = BOTTLENECK_TYPE[view_type]

    # Create lower level view
    low_level_view = view \
        .groupby(list(BOTTLENECK_ORDER[view_type])) \
        .first() \
        .drop(columns=['acc_pat', 'io_cat', 'file_id', 'proc_id'], errors='ignore')

    # Non-proc agg columns
    non_proc_agg_dict = _get_agg_dict(view_types=view_types, view_columns=low_level_view.columns, is_proc=False)
    proc_agg_dict = _get_agg_dict(view_types=view_types, view_columns=low_level_view.columns, is_proc=True)

    # Create mid and high level views
    if bottleneck_type is not 'proc_name':
        mid_level_view = low_level_view \
            .reset_index() \
            .groupby([bottleneck_type, 'proc_name']) \
            .agg(non_proc_agg_dict)

        high_level_view = mid_level_view \
            .reset_index() \
            .groupby([bottleneck_type]) \
            .agg(proc_agg_dict)
    else:
        mid_level_view = low_level_view \
            .reset_index() \
            .groupby([bottleneck_type, 'trange']) \
            .agg(non_proc_agg_dict)

        high_level_view = mid_level_view \
            .reset_index() \
            .groupby([bottleneck_type]) \
            .agg(non_proc_agg_dict)

    low_level_view = low_level_view \
        .map_partitions(set_metric_percentages, metric=metric, max_io_time=max_io_time) \
        .map_partitions(set_metric_scores, metric=metric, col=f"{metric}_pero")

    mid_level_view = mid_level_view \
        .map_partitions(set_metric_percentages, metric=metric, max_io_time=max_io_time) \
        .map_partitions(set_metric_scores, metric=metric, col=f"{metric}_pero")

    high_level_view = high_level_view \
        .map_partitions(set_metric_percentages, metric=metric, max_io_time=max_io_time) \
        .map_partitions(set_metric_scores, metric=metric, col=f"{metric}_pero")

    return dict(
        low_level_view=low_level_view,
        mid_level_view=mid_level_view,
        high_level_view=high_level_view
    )

def _get_agg_dict(view_types: list, view_columns: list, is_proc=False):
    if is_proc:
        agg_dict = {col: max if any(x in col for x in 'duration time'.split()) else sum for col in view_columns}
    else:
        agg_dict = {col: sum for col in view_columns}
#     agg_dict['func_id'] = unique_flatten()
    agg_dict['size_min'] = min
    agg_dict['size_max'] = max
    for view_type in view_types:
        if view_type in agg_dict:
            agg_dict.pop(view_type)
        bottleneck_type = BOTTLENECK_TYPE[view_type]
        if bottleneck_type in agg_dict:
            agg_dict.pop(bottleneck_type)
    return agg_dict


time: 11.8 ms (started: 2023-03-12 14:53:23 -07:00)


In [66]:
bottlenecks, bottleneck_views = detect_bottlenecks(
    views=views,
    view_types=VIEW_TYPES,
)

len(bottlenecks)

15

time: 1min 32s (started: 2023-03-12 14:53:23 -07:00)


In [68]:
for view_key, bottleneck in bottlenecks.items():
    for th in bottleneck.keys():
        print(view_key, th, len(bottleneck[th]))

('trange',) 0.001 40
('trange',) 0.010 0
('trange',) 0.100 0
('trange',) 0.250 0
('trange',) 0.500 0
('trange',) 0.750 0
('file_name',) 0.001 5
('file_name',) 0.010 0
('file_name',) 0.100 0
('file_name',) 0.250 0
('file_name',) 0.500 0
('file_name',) 0.750 0
('proc_name',) 0.001 5
('proc_name',) 0.010 0
('proc_name',) 0.100 0
('proc_name',) 0.250 0
('proc_name',) 0.500 0
('proc_name',) 0.750 0
('trange', 'file_name') 0.001 5
('trange', 'file_name') 0.010 0
('trange', 'file_name') 0.100 0
('trange', 'file_name') 0.250 0
('trange', 'file_name') 0.500 0
('trange', 'file_name') 0.750 0
('trange', 'proc_name') 0.001 5
('trange', 'proc_name') 0.010 0
('trange', 'proc_name') 0.100 0
('trange', 'proc_name') 0.250 0
('trange', 'proc_name') 0.500 0
('trange', 'proc_name') 0.750 0
('file_name', 'trange') 0.001 40
('file_name', 'trange') 0.010 0
('file_name', 'trange') 0.100 0
('file_name', 'trange') 0.250 0
('file_name', 'trange') 0.500 0
('file_name', 'trange') 0.750 0
('file_name', 'proc_name')

In [69]:
max_io_time.compute()

2070387.8

time: 930 ms (started: 2023-03-12 14:55:08 -07:00)


In [70]:
!mkdir -p {log_dir}/bottlenecks

time: 177 ms (started: 2023-03-12 14:55:20 -07:00)


In [71]:
def save_bottlenecks(log_dir: str, bottlenecks: Dict[tuple, object]):
    bottleneck_dir = f"{log_dir}/bottlenecks"
    for view_key, bottleneck_dict in bottlenecks.items():
        file_name = '_'.join(view_key) if isinstance(view_key, tuple) else view_key
        with open(f"{bottleneck_dir}/{file_name}.json", 'w') as json_file:
            json.dump(bottleneck_dict, json_file, cls=NpEncoder, sort_keys=True)

time: 1.52 ms (started: 2023-03-12 14:55:25 -07:00)


In [72]:
save_bottlenecks(log_dir=log_dir, bottlenecks=bottlenecks)

time: 685 ms (started: 2023-03-12 14:55:32 -07:00)
